In [ ]:
# データの永続化
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# TensorFlow 1.Xへの切り替え
%tensorflow_version 1.x

In [ ]:
# CUDAのバージョンの確認
!nvcc --version

In [ ]:
# PyTorch1.7.1のインストール
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# NVIDIA/tacotron2のインストール
!git clone https://github.com/NVIDIA/tacotron2.git
%cd tacotron2
!git submodule init
!git submodule update

In [ ]:
# Apexのインストール
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

In [ ]:
# 依存パッケージのインストール
!pip install -r requirements.txt
!pip install Unidecode==1.0.22

In [ ]:
# work/tacotron2/filelistsフォルダにtranscript_utf8.txtをアップロード

# 学習データと検証データの分割
!head -n 7000 filelists/transcript_utf8.txt > filelists/transcript_train.txt
!tail -n 690 filelists/transcript_utf8.txt > filelists/transcript_val.txt

# work/tacotron2フォルダにwav.zipをアップロード

# 解凍
!unzip wav.zip

In [ ]:
# work/tacotron2フォルダにTacotron2モデルとWaveGlowモデルをアップロード。

# 「hparams.py」の編集。

In [ ]:
# 学習の実行
!python train.py --output_directory=outdir --log_directory=logdir -c tacotron2_statedict.pt --warm_start

In [ ]:
# パッケージのインストール
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

In [ ]:
# plot_data()の準備
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', interpolation='none')

In [ ]:
# ハイパーパラメータの準備
hparams = create_hparams()
hparams.sampling_rate = 22050

In [ ]:
# 先程学習したTacotron2モデルの読み込み
checkpoint_path = "outdir2/checkpoint_4000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

In [ ]:
# WaveGlowモデルの読み込み
waveglow_path = 'waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [ ]:
# テキストを音素に変換
text = "hoNjituwa,seIteNnari."
sequence = np.array(text_to_sequence(text, ['basic_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

In [ ]:
# グラフで確認
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

In [ ]:
# 音声で確認
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)